# Llama 3.2 Vision: Independent Single-Turn Bank Statement Extraction

**Protocol**: Two independent single-turn prompts + Python parsing/filtering

**Key Insight**: Multi-turn conversation degrades accuracy. LLM filtering mixes up rows. Use Python for filtering!

---

## Complete Workflow

```
Turn 0: Image + Prompt → Headers (fresh context)
        ↓ (Python pattern matching)
Turn 1: Image + Prompt → Full Table (fresh context, dynamic markdown example)
        ↓ (Python parsing + filtering)
Schema Fields: TRANSACTION_DATES, LINE_ITEM_DESCRIPTIONS, TRANSACTION_AMOUNTS_PAID
```

### Pipeline Stages:
1. **Turn 0 (LLM)**: Identify column headers from image
2. **Pattern Matching (Python)**: Map headers to concepts (Date, Description, Debit, Credit)
3. **Turn 1 (LLM)**: Extract full markdown table using **dynamic example** matching detected column structure
4. **Python Parsing**: Parse markdown → Filter for debits → Extract schema fields

### Critical Features:
- ❌ **No Turn 2** - LLM filtering mixes up rows!
- ✅ **Python filtering** - Reliable debit/credit separation
- ✅ **Dynamic examples** - Adapt to 3/4/5 column formats
- ✅ **Markdown teaching** - Llama understands markdown format for alignment
- ✅ **Tax accuracy** - Correct Debit/Credit separation critical for identifying purchases

### Why This Works:
- **Turn 0**: Clean context → accurate header identification
- **Turn 1**: Dynamic example + markdown alignment rules → accurate table extraction
- **Python**: Reliable filtering for debit transactions (what taxpayer PAID)

In [1]:
# Cell 1: Imports

from pathlib import Path
import random

import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration

# Set Random Seed for Reproducibility

In [2]:
# Cell 3: Set random seed

from common.reproducibility import set_seed
set_seed(42)

✅ Random seed set to 42 for reproducibility


# Load the model

In [3]:
# Cell 5: Load Llama-3.2-Vision model
# Update this path to your local Llama model
# model_id = "/home/jovyan/shared_PTM/Llama-3.2-11B-Vision-Instruct"
model_id = "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision-Instruct"

print("🔧 Loading Llama-3.2-Vision model...")
# model = MllamaForConditionalGeneration.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# processor = AutoProcessor.from_pretrained(model_id)

from common.llama_model_loader_robust import load_llama_model_robust

model, processor = load_llama_model_robust(
    model_path=model_id,
    use_quantization=False,
    device_map='auto',
    max_new_tokens=2000,
    torch_dtype='bfloat16',
    low_cpu_mem_usage=True,
    verbose=True
)

# Add tie_weights() call
try:
    model.tie_weights()
    print("✅ Model weights tied successfully")
except Exception as e:
    print(f"⚠️ tie_weights() warning: {e}")

# processor

🔧 Loading Llama-3.2-Vision model...


🚀 Loading Llama Vision model with robust multi-GPU optimization...

Features: Smart quantization, memory management, V100 support

🔧 Configuring CUDA memory for Llama...

🔧 CUDA memory allocation configured: max_split_size_mb:64
💡 Using 64MB memory blocks to reduce fragmentation
📊 Initial CUDA state (Multi-GPU Total): Allocated=0.00GB, Reserved=0.00GB


...

🧹 Memory cleanup completed

💾 Final state (Multi-GPU Total): Allocated=21.34GB, Reserved=21.65GB, Fragmentation=0.30GB

🎉 Llama Vision model loading and validation complete!

🔧 Llama optimizations active: 16-bit precision, memory management, vision preservation

✅ Model weights tied successfully


# Load the image

In [4]:
# Cell 7: Load bank statement image
# Update this path to your test image
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_amount_balance.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_date_grouped_cont.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_debit_credit.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_highligted.png"
imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/low_contrast_fixed.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/nab_classic_highligted.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/westpac_debit_credit.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/transaction_summary.png"



# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/images/image_009.png"
print("📁 Loading image...")
image = Image.open(imageName)

# CRITICAL: Store as list for multi-turn compatibility
images = [image]

print(f"✅ Image loaded: {image.size}")
print(f"✅ Images list created with {len(images)} image(s)")

📁 Loading image...
✅ Image loaded: (753, 584)
✅ Images list created with 1 image(s)


# Bank Statement Extraction Protocol
- Turn 0: Identify actual table headers
- Turn 1: Extract full table using dynamic markdown example
- Python: Parse, filter, and extract schema fields

In [5]:
# Cell 9: Turn 0 - Identify table headers (prompt)
# TURN 0: Identify Table Headers
# First, identify the actual column headers used in this specific bank statement

prompt = """
Look at the transaction table in this bank statement image.

IMPORTANT STRUCTURAL NOTE:
Some bank statements show dates as section headings with multiple transactions underneath.
If you see this structure, remember that each transaction needs its explicit date in the final output.

What are the exact column header names used in the transaction table?

List each column header exactly as it appears, in order from left to right.
Do not interpret or rename them - use the EXACT text from the image.
"""

# Create message structure for Llama
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": prompt,
            },
        ],
    }
]

print("💬 TURN 0: Identifying actual table headers")
print("🤖 Generating response with Llama-3.2-Vision...")

💬 TURN 0: Identifying actual table headers
🤖 Generating response with Llama-3.2-Vision...


# Process the prompt

In [6]:
# Cell 11: Turn 0 - Execute and parse headers
# Process the input using the CORRECT multi-turn pattern
# Based on: https://medium.com/data-science/chat-with-your-images-using-multimodal-llms-60af003e8bfa

textInput = processor.apply_chat_template(
    messageDataStructure, add_generation_prompt=True
)

# CRITICAL: Use named parameter 'images=' with list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response with deterministic parameters
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# CRITICAL: Trim input tokens from output (this is the key to clean responses!)
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("✅ Response generated successfully!")
print("\n" + "=" * 60)
print("TURN 0 - IDENTIFIED TABLE HEADERS:")
print("=" * 60)
print(cleanedOutput)
print("=" * 60)

# CRITICAL: Parse the identified headers for use in subsequent turns
# Extract column names from the response
header_lines = [line.strip() for line in cleanedOutput.split('\n') if line.strip()]
identified_headers = []

# Look for numbered list or bullet points
for line in header_lines:
    # Remove common list markers
    cleaned = line.lstrip('0123456789.-•* ').strip()
    
    # Strip markdown bold formatting
    cleaned = cleaned.replace('**', '').replace('__', '')
    
    # Skip section headers (lines ending with colon)
    if cleaned.endswith(':'):
        continue
    
    # Skip long sentences (likely explanatory text, not headers)
    if len(cleaned) > 40:
        continue
        
    if cleaned and len(cleaned) > 2:  # Ignore very short strings
        identified_headers.append(cleaned)

print(f"\n📋 Parsed {len(identified_headers)} column headers:")
for i, header in enumerate(identified_headers, 1):
    print(f"  {i}. '{header}'")

# Store headers for use in subsequent turns
table_headers = identified_headers

# Save the table headers
output_path = Path("llama_table_headers.txt")
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput)

print(f"\n✅ Table headers saved to: {output_path}")
print("💡 These LITERAL header names will be used in Turn 1 & 2 prompts")

✅ Response generated successfully!

TURN 0 - IDENTIFIED TABLE HEADERS:
**Transaction Table Column Headers:**

*   **Date**
*   **Description**
*   **Credits**
*   **Debits**

📋 Parsed 4 column headers:
  1. 'Date'
  2. 'Description'
  3. 'Credits'
  4. 'Debits'

✅ Table headers saved to: llama_table_headers.txt
💡 These LITERAL header names will be used in Turn 1 & 2 prompts


## Pattern Matching: Map Generic Concepts to Actual Headers

Different bank statements use different column names. Use pattern matching to identify:
- Which header represents **Date**
- Which header represents **Description/Details**  
- Which header represents **Debit/Withdrawal**

In [7]:
# Cell 13: Pattern Matching - Map headers to generic columns
# Pattern Matching: Map extracted headers to generic concepts
# This handles variety in bank statement column naming conventions

# Pattern keywords for each concept (in priority order)
DATE_PATTERNS = ['date', 'day', 'transaction date', 'trans date']
DESCRIPTION_PATTERNS = [
    'description', 'details', 'transaction details', 'trans details',
    'particulars', 'narrative', 'transaction', 'trans'
]
DEBIT_PATTERNS = ['debit', 'withdrawal', 'withdrawals', 'paid', 'paid out', 'spent', 'dr']
CREDIT_PATTERNS = ['credit', 'deposit', 'deposits', 'received', 'cr']
BALANCE_PATTERNS = ['balance', 'bal', 'running balance']

# NEW: Pattern for single-column transaction formats (e.g., "Amount" instead of separate Debit/Credit)
AMOUNT_PATTERNS = ['amount', 'amt', 'value', 'total']

def match_header(headers, patterns, fallback=None):
    """Match a header using pattern keywords.
    
    Matching strategy:
    1. Exact match (case-insensitive)
    2. Substring match (only for patterns with length > 2 to avoid false positives)
    """
    headers_lower = [h.lower() for h in headers]
    
    # Try exact match first
    for pattern in patterns:
        for i, header_lower in enumerate(headers_lower):
            if pattern == header_lower:
                return headers[i]
    
    # Try substring match (only for patterns longer than 2 chars)
    for pattern in patterns:
        if len(pattern) > 2:  # Avoid false positives like 'cr' matching 'description'
            for i, header_lower in enumerate(headers_lower):
                if pattern in header_lower:
                    return headers[i]
    
    return fallback

# Perform pattern matching on extracted headers
date_col = match_header(table_headers, DATE_PATTERNS, fallback=table_headers[0] if table_headers else 'Date')
desc_col = match_header(table_headers, DESCRIPTION_PATTERNS, fallback=table_headers[1] if len(table_headers) > 1 else 'Description')

# NEW: First try to match a generic "Amount" column (for 4-column formats)
amount_col = match_header(table_headers, AMOUNT_PATTERNS, fallback=None)

# Use amount_col as fallback if no separate debit/credit columns exist
# This handles formats like: Date | Description | Amount | Balance
debit_col = match_header(table_headers, DEBIT_PATTERNS, fallback=amount_col if amount_col else 'Debit')
credit_col = match_header(table_headers, CREDIT_PATTERNS, fallback=amount_col if amount_col else 'Credit')
balance_col = match_header(table_headers, BALANCE_PATTERNS, fallback='Balance')

print("=" * 60)
print("PATTERN MATCHING RESULTS:")
print("=" * 60)
print(f"📋 Extracted Headers: {table_headers}")
print(f"\n🔍 Mapped Columns:")
print(f"  Date        → '{date_col}'")
print(f"  Description → '{desc_col}'")
print(f"  Debit       → '{debit_col}'")
print(f"  Credit      → '{credit_col}'")
print(f"  Balance     → '{balance_col}'")
if amount_col:
    print(f"\n💡 Single-column format detected: '{amount_col}' used for both debit and credit")
print("=" * 60)
print("\n✅ These literal column names will be used in Turn 1 and Turn 2")
print("💡 Adjust patterns above if matching fails for your bank statement format")

PATTERN MATCHING RESULTS:
📋 Extracted Headers: ['Date', 'Description', 'Credits', 'Debits']

🔍 Mapped Columns:
  Date        → 'Date'
  Description → 'Description'
  Debit       → 'Debits'
  Credit      → 'Credits'
  Balance     → 'Balance'

✅ These literal column names will be used in Turn 1 and Turn 2
💡 Adjust patterns above if matching fails for your bank statement format


### 🔑 Independent Single-Turn Pattern (NOT Multi-Turn Conversation)

**CRITICAL INSIGHT**: Multi-turn conversation accumulates context and degrades accuracy.

We use **two independent single-turn prompts**, each with fresh context:

#### Key Principles:

1. **No Conversation History**: Each turn is completely independent
2. **Fresh Image Attention**: Each turn processes the image directly
3. **No Context Accumulation**: Prevents attention dilution
4. **Headers as Parameters**: Turn 0 headers used to generate dynamic examples for Turn 1
5. **Python Filtering**: LLM filtering mixes up rows - Python is reliable

#### Message Structure for Each Turn:

Every turn uses fresh structure:
```python
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "<prompt with dynamic example>"}
        ]
    }
]
```

**No assistant responses in history. No conversation accumulation.**

#### Why This Works Better:

- **Turn 0**: Clean context → accurate header identification
- **Turn 1**: Clean context + dynamic example → accurate table extraction  
- **Python**: Reliable parsing and filtering (no row mixing!)

Each turn has **full attention** on the image, not diluted by conversation history.

In [8]:
# Cell 15: NO conversation history (independent turns)
# 
# CRITICAL: We do NOT use conversation history in this notebook.
# Each turn is completely independent with fresh context.
#
# Why? Multi-turn conversation accumulates context and degrades accuracy:
# - Turn 0: ~50 tokens → accurate
# - Turn 1 with history: ~350 tokens → attention diluted → less accurate
# - Turn 2 with history: ~2000 tokens → attention heavily diluted → row mixing!
#
# Instead: 
# - Turn 0: Fresh context → headers
# - Turn 1: Fresh context + dynamic example → full table
# - Python: Parse and filter (no LLM confusion!)

print("✅ Independent turn approach - NO conversation history")
print("💡 Each turn has fresh context with direct image access")
print("🐍 Python handles all filtering - no LLM row mixing!")

✅ Independent turn approach - NO conversation history
💡 Each turn has fresh context with direct image access
🐍 Python handles all filtering - no LLM row mixing!


### TURN 1: Extract Full Table in Markdown

Now that we know the actual column headers, extract the complete table:

In [9]:
# Cell 17: Turn 1 - Extract full table (INDEPENDENT, fresh context)

# Build the header string using LITERAL names from Turn 0
header_string = " | ".join(table_headers)
separator_row = " | ".join(["---"] * len(table_headers))

# Build dynamic example rows based on detected column structure
# CRITICAL: Examples must emphasize correct Debit/Credit column alignment!
def build_dynamic_example(headers, date_col, desc_col, debit_col, credit_col, balance_col):
    """Generate example rows matching detected column structure.
    
    Emphasizes correct Debit/Credit alignment for tax purposes:
    - Debits (purchases/withdrawals) = money OUT = what taxpayer PAID
    - Credits (deposits/income) = money IN = NOT purchases
    """
    
    # Check if we have separate debit/credit columns
    has_separate_debit_credit = (debit_col in headers and credit_col in headers and debit_col != credit_col)
    
    rows = []
    
    if has_separate_debit_credit:
        # 5-column format: Emphasize Debit vs Credit with clear examples
        
        # Example 1: DEBIT transaction (purchase/withdrawal) - money OUT
        row1 = []
        for h in headers:
            if h == date_col:
                row1.append("15 Jan")
            elif h == desc_col:
                row1.append("ATM Withdrawal City Branch")
            elif h == debit_col:
                row1.append("200.00")  # Amount in DEBIT column
            elif h == credit_col:
                row1.append("")  # Credit column EMPTY
            elif h == balance_col:
                row1.append("$1,500.00 CR")
            else:
                row1.append("")
        rows.append(" | ".join(row1))
        
        # Example 2: CREDIT transaction (deposit) - money IN
        row2 = []
        for h in headers:
            if h == date_col:
                row2.append("16 Jan")
            elif h == desc_col:
                row2.append("Salary Employer Name Ref 12345")
            elif h == debit_col:
                row2.append("")  # Debit column EMPTY
            elif h == credit_col:
                row2.append("3,500.00")  # Amount in CREDIT column
            elif h == balance_col:
                row2.append("$5,000.00 CR")
            else:
                row2.append("")
        rows.append(" | ".join(row2))
        
        # Example 3: Another DEBIT transaction (purchase)
        row3 = []
        for h in headers:
            if h == date_col:
                row3.append("17 Jan")
            elif h == desc_col:
                row3.append("Online Purchase Store Name")
            elif h == debit_col:
                row3.append("150.00")  # Amount in DEBIT column
            elif h == credit_col:
                row3.append("")  # Credit column EMPTY
            elif h == balance_col:
                row3.append("$4,850.00 CR")
            else:
                row3.append("")
        rows.append(" | ".join(row3))
        
    else:
        # 4-column format: Single Amount column
        row1 = []
        for h in headers:
            if h == date_col:
                row1.append("15 Jan")
            elif h == desc_col:
                row1.append("ATM Withdrawal City Branch")
            elif h == debit_col:  # This is the Amount column
                row1.append("200.00")
            elif h == balance_col:
                row1.append("$1,500.00 CR")
            else:
                row1.append("")
        rows.append(" | ".join(row1))
        
        row2 = []
        for h in headers:
            if h == date_col:
                row2.append("16 Jan")
            elif h == desc_col:
                row2.append("Salary Employer Name Ref 12345")
            elif h == debit_col:  # This is the Amount column
                row2.append("3,500.00")
            elif h == balance_col:
                row2.append("$5,000.00 CR")
            else:
                row2.append("")
        rows.append(" | ".join(row2))
    
    return rows

# Generate dynamic example rows
example_rows = build_dynamic_example(table_headers, date_col, desc_col, debit_col, credit_col, balance_col)

# Build complete example table
example_table = f"""| {header_string} |
| {separator_row} |
""" + "\n".join([f"| {row} |" for row in example_rows])

follow_up_prompt = f"""
Extract ONLY the MAIN transaction table from this bank statement image in markdown format.

Here is an example showing the EXACT format and column alignment I want:

{example_table}

CRITICAL EXTRACTION RULES:
1. EACH ROW MUST be ALIGNED UNDER the CORRECT COLUMN HEADING
2. If you see a transaction row that looks like:
      a date |  line 1  |  | amount1 | amount2
             |  line 2  |  |         |
    you must extract it as:
      a date |  line 1 line 2 |  | amount1 | amount2
3. If you see a transaction row that looks like:
      a date |  line 1  | amount1 |  | amount2
             |  line 2  |         |  |
    you must extract it as:
      a date |  line 1 line 2 | amount1 |  | amount2
4. Look at which column each amount appears under in the image - put it in that SAME column in your table
5. If a column is empty in the image, leave it empty in your table (spaces between pipes: |  |)
6. IGNORE ANY OTHER TABLES that may exist (like summary tables or interest rate tables)
7. DO NOT ADD ANY further information or explanation - ONLY output the markdown table

"""

# CRITICAL: Create FRESH message structure (NOT appending to conversation history)
messageDataStructure_turn1 = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": follow_up_prompt}
        ]
    }
]

print(f"💬 TURN 1: Extract full markdown table (INDEPENDENT)")
print(f"📋 Detected headers: {table_headers}")
print(f"💡 Dynamic example generated - emphasizing Debit/Credit column alignment")
print(f"🔄 Fresh context (no conversation history)")
print("🤖 Generating response with Llama-3.2-Vision...")

# Process with FRESH context
textInput = processor.apply_chat_template(
    messageDataStructure_turn1, add_generation_prompt=True
)

# CRITICAL: Use named parameter 'images=' with list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# CRITICAL: Trim input tokens from output
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput2 = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("\n✅ Follow-up response generated successfully!")
print("\n" + "=" * 60)
print("TURN 1 - FULL MARKDOWN TABLE:")
print("=" * 60)
print(cleanedOutput2)
print("=" * 60)

# Save the markdown table
output_path = Path("llama_markdown_table_extraction.txt")
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput2)

print(f"\n✅ Markdown table saved to: {output_path}")
print("💡 Ready for Python parsing and filtering")

💬 TURN 1: Extract full markdown table (INDEPENDENT)
📋 Detected headers: ['Date', 'Description', 'Credits', 'Debits']
💡 Dynamic example generated - emphasizing Debit/Credit column alignment
🔄 Fresh context (no conversation history)
🤖 Generating response with Llama-3.2-Vision...

✅ Follow-up response generated successfully!

TURN 1 - FULL MARKDOWN TABLE:
| Date | Description | Credits | Debits |
| --- | --- | --- | --- |
| 05 Sep 2023 | Otr Supermarket 9876 Sydney Aus |  | 45.80 |
| 06 Sep 2023 | Otr Convenience 4321 Melbourne Aus |  | 28.45 |
| 06 Sep 2023 | Otr Convenience 4321 Melbourne Aus |  | 15.90 |
| 07 Sep 2023 | Retailstore 8765 Perth Aus |  | 12.35 |
| 07 Sep 2023 | Otr Convenience 4321 Melbourne Aus |  | 34.60 |
| 07 Sep 2023 | Valley Shopping Ct Brisbane Aus |  | 67.00 |
| 07 Sep 2023 | Plaza Hotel Sydney Sydney Cre Aus | 89.00 |  |
| 07 Sep 2023 | Plaza Hotel Sydney Sydney Cre Aus | 23.50 |  |
| 07 Sep 2023 | Department Store Pl Adelaide Aus | 156.95 |  |
| 07 Sep 2023 | Pl

## Python Parsing and Filtering

Parse the Turn 1 markdown table, filter for debit transactions, and extract schema fields using Python.

In [10]:
# Cell 19: Parse Turn 1 markdown table and filter for debits (Python)
import re
from datetime import datetime

def parse_markdown_table(markdown_text):
    """Parse markdown table into list of dictionaries.
    
    CRITICAL: Must preserve empty columns for correct Debit/Credit alignment!
    """
    lines = [line.strip() for line in markdown_text.strip().split('\n') if line.strip()]
    
    # Find header row (first line with pipes)
    header_idx = None
    for i, line in enumerate(lines):
        if '|' in line:
            # Skip separator rows (contain only pipes, hyphens, and spaces)
            cleaned = line.replace('|', '').replace('-', '').replace(' ', '')
            if cleaned:  # Has actual content, not just separators
                header_idx = i
                break
    
    if header_idx is None:
        return []
    
    # Parse headers - KEEP empty values to preserve column positions
    header_line = lines[header_idx]
    header_parts = [h.strip() for h in header_line.split('|')]
    # Remove leading/trailing empty strings from pipe delimiters
    if header_parts and header_parts[0] == '':
        header_parts = header_parts[1:]
    if header_parts and header_parts[-1] == '':
        header_parts = header_parts[:-1]
    # Filter out any remaining empty headers
    headers = [h for h in header_parts if h]
    
    print(f"🔍 Debug: Parsed {len(headers)} headers: {headers}")
    
    # Parse data rows (skip header and separator)
    rows = []
    for idx, line in enumerate(lines[header_idx + 1:], start=header_idx+1):
        if '|' not in line:
            continue
            
        # Skip separator rows
        cleaned = line.replace('|', '').replace('-', '').replace(' ', '')
        if not cleaned:
            continue
        
        # Parse values - KEEP empty values to preserve column positions!
        value_parts = [v.strip() for v in line.split('|')]
        # Remove leading/trailing empty strings from pipe delimiters
        if value_parts and value_parts[0] == '':
            value_parts = value_parts[1:]
        if value_parts and value_parts[-1] == '':
            value_parts = value_parts[:-1]
        
        print(f"🔍 Debug row {idx}: {len(value_parts)} values: {value_parts}")
        
        # Match to headers length
        if len(value_parts) == len(headers):
            rows.append(dict(zip(headers, value_parts)))
        else:
            print(f"⚠️  Row {idx} mismatch: {len(value_parts)} values vs {len(headers)} headers - SKIPPED")
    
    return rows

def filter_debit_transactions(rows, debit_col):
    """Filter rows to only those with debit (purchase) amounts.
    
    CRITICAL: For tax purposes, we only want transactions where taxpayer PAID money (debits).
    """
    debit_rows = []
    for row in rows:
        debit_value = row.get(debit_col, '').strip()
        # Include row if debit column has a value (not empty)
        if debit_value:
            debit_rows.append(row)
    
    return debit_rows

def extract_schema_fields(rows, date_col, desc_col, debit_col):
    """Extract fields in universal.yaml schema format."""
    if not rows:
        return {
            'TRANSACTION_DATES': 'NOT_FOUND',
            'LINE_ITEM_DESCRIPTIONS': 'NOT_FOUND',
            'TRANSACTION_AMOUNTS_PAID': 'NOT_FOUND',
            'STATEMENT_DATE_RANGE': 'NOT_FOUND'
        }
    
    # Extract lists
    dates = []
    descriptions = []
    amounts = []
    
    for row in rows:
        date = row.get(date_col, '').strip()
        desc = row.get(desc_col, '').strip()
        amount = row.get(debit_col, '').strip()
        
        if date:
            dates.append(date)
        if desc:
            descriptions.append(desc)
        if amount:
            amounts.append(amount)
    
    # Calculate statement date range - use literal date format from image
    # No parsing, no year assumption - just "earliest date - latest date"
    date_range = 'NOT_FOUND'
    if dates:
        # Use first and last date as-is (same format as in the image)
        date_range = f"{dates[0]} - {dates[-1]}"
    
    return {
        'TRANSACTION_DATES': ' | '.join(dates) if dates else 'NOT_FOUND',
        'LINE_ITEM_DESCRIPTIONS': ' | '.join(descriptions) if descriptions else 'NOT_FOUND',
        'TRANSACTION_AMOUNTS_PAID': ' | '.join(amounts) if amounts else 'NOT_FOUND',
        'STATEMENT_DATE_RANGE': date_range
    }

print("=" * 60)
print("PARSING TURN 1 MARKDOWN TABLE:")
print("=" * 60)

# Parse the full markdown table from Turn 1
all_rows = parse_markdown_table(cleanedOutput2)

print(f"\n📊 Parsed {len(all_rows)} total transactions from Turn 1 markdown table")

if all_rows:
    # Show sample parsed row
    print(f"\n🔍 Sample parsed row:")
    for key, value in all_rows[0].items():
        print(f"  {key}: '{value}'")

# Filter to only debit (purchase) transactions - Python filtering, not LLM!
debit_rows = filter_debit_transactions(all_rows, debit_col)

print(f"\n💰 Filtered to {len(debit_rows)} debit transactions (taxpayer purchases)")
print("\n" + "=" * 60)
print("DEBIT TRANSACTIONS (WHAT TAXPAYER PAID):")
print("=" * 60)
for i, row in enumerate(debit_rows, 1):
    print(f"\nTransaction {i}:")
    print(f"  {date_col}: {row.get(date_col, '')}")
    print(f"  {desc_col}: {row.get(desc_col, '')}")
    print(f"  {debit_col}: {row.get(debit_col, '')}")

# Extract schema fields using the LITERAL column names from pattern matching
schema_fields = extract_schema_fields(debit_rows, date_col, desc_col, debit_col)

print("\n" + "=" * 60)
print("EXTRACTED SCHEMA FIELDS (TAX-RELEVANT DATA):")
print("=" * 60)
for field, value in schema_fields.items():
    print(f"{field}: {value}")
print("=" * 60)

# Save to file
output_path = Path("llama_extracted_fields.txt")
with output_path.open("w", encoding="utf-8") as f:
    for field, value in schema_fields.items():
        f.write(f"{field}: {value}\n")

print(f"\n✅ Schema fields saved to: {output_path}")
print(f"💡 Fields extracted from columns: '{date_col}' | '{desc_col}' | '{debit_col}'")
print(f"🎯 Success: Python parsing + filtering from Turn 1 markdown table")

PARSING TURN 1 MARKDOWN TABLE:
🔍 Debug: Parsed 4 headers: ['Date', 'Description', 'Credits', 'Debits']
🔍 Debug row 2: 4 values: ['05 Sep 2023', 'Otr Supermarket 9876 Sydney Aus', '', '45.80']
🔍 Debug row 3: 4 values: ['06 Sep 2023', 'Otr Convenience 4321 Melbourne Aus', '', '28.45']
🔍 Debug row 4: 4 values: ['06 Sep 2023', 'Otr Convenience 4321 Melbourne Aus', '', '15.90']
🔍 Debug row 5: 4 values: ['07 Sep 2023', 'Retailstore 8765 Perth Aus', '', '12.35']
🔍 Debug row 6: 4 values: ['07 Sep 2023', 'Otr Convenience 4321 Melbourne Aus', '', '34.60']
🔍 Debug row 7: 4 values: ['07 Sep 2023', 'Valley Shopping Ct Brisbane Aus', '', '67.00']
🔍 Debug row 8: 4 values: ['07 Sep 2023', 'Plaza Hotel Sydney Sydney Cre Aus', '89.00', '']
🔍 Debug row 9: 4 values: ['07 Sep 2023', 'Plaza Hotel Sydney Sydney Cre Aus', '23.50', '']
🔍 Debug row 10: 4 values: ['07 Sep 2023', 'Department Store Pl Adelaide Aus', '156.95', '']
🔍 Debug row 11: 4 values: ['07 Sep 2023', 'Plaza Motorcycl Brisbane Aus', '245.00', '